<a href="https://colab.research.google.com/github/royd2023/Book-Recommendation-KNN/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-22 18:44:22--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.3’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.08s   

2025-07-22 18:44:22 (323 MB/s) - ‘book-crossings.zip.3’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [7]:
print(f"# users: {df_ratings['user'].nunique()}")
print(f"# books: {df_ratings['isbn'].nunique()}")

# users: 105283
# books: 340556


In [46]:
# Step 1: Filter popular books
book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index

# Step 2: Filter active users
user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index

# Step 3: Apply both filters
df_filtered = df_ratings[
    df_ratings['isbn'].isin(popular_books) &
    df_ratings['user'].isin(active_users)
]

In [47]:
print(f"Filtered users: {df_filtered['user'].nunique()}")
print(f"Filtered books: {df_filtered['isbn'].nunique()}")

Filtered users: 888
Filtered books: 731


Make a utility matrix

In [48]:
util_matrix = df_filtered.pivot(
    index = 'user',
    columns='isbn',
    values='rating'
)

In [56]:
# transpose to get vectors, where each vector has all user ratings for a book
book_matrix = util_matrix.transpose()

# fills in NaNs with 0
book_matrix_filled = book_matrix.fillna(0)

# from sklearn.preprocessing import normalize
# book_matrix_normalized = normalize(book_matrix_filled, norm='l2', axis=1)
# book_matrix_normalized = pd.DataFrame(book_matrix_normalized,
#                                       index=book_matrix_filled.index,
#                                       columns=book_matrix_filled.columns)

In [57]:
model = NearestNeighbors(algorithm='brute', metric ='cosine')
model.fit(book_matrix_filled)

NearestNeighbors(algorithm='brute', metric='cosine')

In [70]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    isbn_match = df_books[df_books['title'].str.lower() == book.lower()]['isbn']
    if isbn_match.empty:
        return f"Book not found: {book}"
    isbn = isbn_match.values[0]

    if isbn not in book_matrix_filled.index:
        return f"ISBN not found: {isbn}"

    book_vector = book_matrix_filled.loc[isbn].values.reshape(1, -1)
    distances, indices = model.kneighbors(book_vector, n_neighbors=6)

    recommended_isbns = [book_matrix_filled.index[i] for i in indices[0][1:6]]
    recommended_sim = [dist for dist in distances[0][1:6]]

    recommended_books = []
    for i, isbn_rec in enumerate(recommended_isbns):
        title = df_books[df_books['isbn'] == isbn_rec]['title'].values[0]
        recommended_books.append([title, recommended_sim[i]])

    recommended_books.reverse()

    return [book, recommended_books]

In [71]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

  # Check if the function returned an error message
  if isinstance(recommends, str):
      print(f"Recommendation failed: {recommends}")
      test_pass = False
      return # Exit the function if recommendation failed

  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(min(len(recommends[1]), len(recommended_books))): # Iterate up to the minimum length
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
